In [20]:
import pandas as pd
import datetime as dt
import numpy as np

In [21]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/ema_mio202537.csv")

In [ ]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "Semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo)
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

ema_def["coef_ventas_norm"] = ema_def["coef_ventas"] / ema_def["coef_ventas"].max()

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%)
conf_acumulado_cantidad = 10
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.799:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [54]:
ema_ord = ema_def[["store_name","product_ref","Año","Semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

In [55]:
bq.head((10))

,store_name,product_ref,Año,Semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
2575,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,37,38.554918,13507.71,520788.65,0.0117,0.2194,98949.8,0.19,0.03,3.935484,0.19,1.0,0.28,AAA
2047,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,37,33.981416,32797.97,1114521.46,0.0251,0.1324,122597.4,0.11,0.05,1.585421,0.30,0.8,0.22,AAA
3184,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,37,30.869047,9774.98,301744.32,0.0068,0.4198,190098.9,0.63,0.08,5.621262,0.93,1.0,0.37,AAA
2639,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,37,30.471784,10680.97,325468.21,0.0073,0.4130,55329.6,0.17,0.10,3.096774,1.10,1.0,0.19,AAA
2598,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,37,25.581835,11371.65,290907.67,0.0065,0.4529,31999.8,0.11,0.12,2.157303,1.21,0.8,0.22,AAA
964,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,37,23.910276,20638.72,493477.49,0.0111,0.2532,78956.4,0.16,0.13,2.307692,1.37,0.8,0.17,AAA
1361,SUCURSAL BARRANQUILLA,BCS10035125,2025.0,37,22.682642,20956.60,475351.06,0.0107,0.2749,66549.1,0.14,0.15,2.476190,1.51,0.8,0.19,AAA
947,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,37,19.326151,31058.67,600244.55,0.0135,0.1958,48019.6,0.08,0.16,1.483636,1.59,0.5,0.17,AAA
1080,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,37,19.151535,27520.25,527055.03,0.0119,0.2077,57976.1,0.11,0.18,1.811321,1.70,0.8,0.19,AAA
2746,SUCURSAL BARRANQUILLA,DAB02982025,2025.0,37,18.685074,48785.61,911562.73,0.0205,0.1529,191428.2,0.21,0.19,3.172414,1.91,1.0,0.18,AAA


In [56]:
df_real = pd.read_csv("/home/donsson/proyectos/API/real_ema.csv")

In [66]:
df_real= df_real[["store_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]
df_mio = ema_ord[["store_name","product_ref","Semana","Ema","acumulado_combinado","Clasificacion"]]

In [70]:
df_real.groupby("store_name")["clasificacion"].count() 

store_name
PRINCIPAL COTA           4862
SUCURSAL BARRANQUILLA    4862
SUCURSAL BUCARAMANGA     4862
SUCURSAL CALI            4862
SUCURSAL CALLE 6         4862
SUCURSAL MEDELLIN        9724
SUCURSAL NORTE           4862
SUCURSAL VALLADOLID      4862
Name: clasificacion, dtype: int64

In [72]:
df_mio.groupby("store_name")["Clasificacion"].count() 

store_name
0                        3927
PRINCIPAL COTA            178
SUCURSAL BARRANQUILLA     387
SUCURSAL BUCARAMANGA      271
SUCURSAL CALI             154
SUCURSAL CALLE 6          423
SUCURSAL MEDELLIN         290
SUCURSAL NORTE            166
SUCURSAL VALLADOLID       209
Name: Clasificacion, dtype: int64

In [ ]:
df_mio.query("product_ref=='BAE05001125'")

,store_name,product_ref,Semana,Ema,acumulado_combinado,Clasificacion
528,0,BAE05001125,0,0.0,NaN,C


In [75]:
df_real.query("product_name=='BAE05001125'")

,store_name,product_name,semana,ema,acumulado_combinado,clasificacion
862,SUCURSAL BARRANQUILLA,BAE05001125,37,0.16,0.975169,C
7534,SUCURSAL MEDELLIN,BAE05001125,37,0.00,1.000000,C
12987,SUCURSAL MEDELLIN,BAE05001125,37,0.00,1.000000,C
17798,SUCURSAL CALI,BAE05001125,37,0.00,1.000029,C
22589,PRINCIPAL COTA,BAE05001125,37,0.00,1.000180,C
27619,SUCURSAL CALLE 6,BAE05001125,37,0.00,1.000211,C
32331,SUCURSAL VALLADOLID,BAE05001125,37,0.00,1.000000,C
37283,SUCURSAL BUCARAMANGA,BAE05001125,37,0.00,1.000000,C
42035,SUCURSAL NORTE,BAE05001125,37,0.00,1.000115,C
